In [1]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#read csv file
data = pd.read_csv('Seattle_Real_Time_Fire_911_Calls.csv')

In [2]:
#print first and last rows of the data
print(data.head())

                 Address          Type                Datetime   Latitude  \
0             904 7th Av      Car Fire  05/22/2019 12:55:00 PM  47.606900   
1         9610 53rd Av S  Aid Response  05/22/2019 12:55:00 PM  47.515984   
2            106 Pike St  Aid Response  05/22/2019 12:57:00 PM  47.608849   
3            1902 2nd Av  Aid Response  07/01/2019 07:45:00 AM  47.610972   
4  4737 Roosevelt Way Ne      Unk Odor  07/01/2019 07:45:00 AM  47.663338   

    Longitude                Report Location Incident Number  
0 -122.329051    POINT (-122.329051 47.6069)      F190051945  
1 -122.266529  POINT (-122.266529 47.515984)      F190051946  
2 -122.339945  POINT (-122.339945 47.608849)      F190051950  
3 -122.340465  POINT (-122.340465 47.610972)      F190068848  
4 -122.317421  POINT (-122.317421 47.663338)      F190068849  


In [3]:
# Get a summary of the dataset
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838057 entries, 0 to 838056
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Address          838050 non-null  object 
 1   Type             838053 non-null  object 
 2   Datetime         838053 non-null  object 
 3   Latitude         838048 non-null  float64
 4   Longitude        838048 non-null  float64
 5   Report Location  838048 non-null  object 
 6   Incident Number  838052 non-null  object 
dtypes: float64(2), object(5)
memory usage: 44.8+ MB
None


In [4]:
# Check basic statistics of the numerical variables
print(data.describe())

            Latitude      Longitude
count  838048.000000  838048.000000
mean       47.620251    -122.330887
std         0.055893       0.031763
min        47.258946    -122.454987
25%        47.587553    -122.349576
50%        47.613831    -122.330457
75%        47.663074    -122.311456
max        47.776695    -122.031332


In [5]:
# Check for missing values
print(data.isnull().sum())
data.dropna()

Address            7
Type               4
Datetime           4
Latitude           9
Longitude          9
Report Location    9
Incident Number    5
dtype: int64


,Address,Type,Datetime,Latitude,Longitude,Report Location,Incident Number
0,904 7th Av,Car Fire,05/22/2019 12:55:00 PM,47.606900,-122.329051,POINT (-122.329051 47.6069),F190051945
1,9610 53rd Av S,Aid Response,05/22/2019 12:55:00 PM,47.515984,-122.266529,POINT (-122.266529 47.515984),F190051946
2,106 Pike St,Aid Response,05/22/2019 12:57:00 PM,47.608849,-122.339945,POINT (-122.339945 47.608849),F190051950
3,1902 2nd Av,Aid Response,07/01/2019 07:45:00 AM,47.610972,-122.340465,POINT (-122.340465 47.610972),F190068848
4,4737 Roosevelt Way Ne,Unk Odor,07/01/2019 07:45:00 AM,47.663338,-122.317421,POINT (-122.317421 47.663338),F190068849
...,...,...,...,...,...,...,...
838047,500 5th Av,Medic Response,02/02/2014 02:22:49 PM,47.602985,-122.328535,POINT (-122.328535 47.602985),F140011854
838048,5th Av S / S Cloverdale St,Motor Vehicle Accident,02/02/2014 02:24:05 PM,47.526481,-122.328199,POINT (-122.328199 47.526481),F140011855
838049,1401 Broadway,Aid Response,02/02/2014 02:28:42 PM,47.612963,-122.320762,POINT (-122.320762 47.612963),F140011856
838050,2121 26th Av S,Auto Fire Alarm,02/02/2014 02:30:26 PM,47.584234,-122.299024,POINT (-122.299024 47.584234),F140011858


In [ ]:
sns.histplot(data['Datetime'], kde=True)
plt.show()

In [ ]:
sns.histplot(data['Latitude'], kde=True)
plt.show()

In [ ]:
sns.histplot(data['Longitude'], kde=True)
plt.show()

In [ ]:
sns.boxplot(x='Latitude', data=data)
plt.show()

In [ ]:
sns.boxplot(x='Longitude', data=data)
plt.show()

In [ ]:
plt.scatter(data['Latitude'], data['Longitude'])
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

In [ ]:
# convert to datetime
data["Datetime"] = pd.to_datetime(data["Datetime"])
data["DayOfWeek"] = data["Datetime"].dt.weekday
data["IsWeekend"] = data["DayOfWeek"] >= 5

# Count incidents by weekday/weekend
incident_counts = data.groupby('IsWeekend')['Incident Number'].count()

# Create a bar plot
sns.barplot(x=incident_counts.index, y=incident_counts.values)
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.xlabel('Day Type')
plt.ylabel('Incident Count')
plt.title('Incidents on Weekdays vs. Weekends')
plt.show()

In [ ]:
#top regions
top_regions_total = data['Address'].value_counts().head(5)
#top jurisdictions
top_regions_jurisdiction = data.groupby('Address')['Incident Number'].count().nlargest(5)
#incidents per year
data['Year'] = pd.to_datetime(data['Datetime']).dt.year
top_regions_yearly = data.groupby(['Year', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per month
data['Month'] = pd.to_datetime(data['Datetime']).dt.month
top_regions_monthly = data.groupby(['Month', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per week
#data['Week'] = pd.to_datetime(data['Datetime']).dt.week
#top_regions_weekly = df.groupby(['Week', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per day
data['Day'] = pd.to_datetime(data['Datetime']).dt.date
top_regions_daily = data.groupby(['Day', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)

In [ ]:
print(top_regions_daily .head())

In [ ]:
print(top_regions_yearly .head())

In [ ]:
print(top_regions_monthly.head())

In [ ]:
#popular incidents on a daily basis
daily_popular_incidents = data.groupby(data['Datetime'].dt.date)['Type'].value_counts().groupby(level=0).idxmax()
print(daily_popular_incidents.head())

In [ ]:
#popular incidents on weekdays and weekends
weekday_popular_incidents = data[data['IsWeekend'] == False]['Type'].value_counts().idxmax()
weekend_popular_incidents = data[data['IsWeekend'] == True]['Type'].value_counts().idxmax()
print(weekday_popular_incidents)
print(weekend_popular_incidents)

In [ ]:
# regions that qualify for mobile emergency patrols
data['Week'] = data['Datetime'].dt.strftime('%Y-%U')
incident_counts = data.groupby(['Address', 'Type', 'Week']).size().reset_index(name='Count')
qualified_regions = incident_counts[incident_counts['Count'] > 50]['Address'].unique()
print(qualified_regions.tolist())

In [ ]:
#how much revenue the private company makes for every year of its operation
medevac_counts = (data['Type'] == 'Medic Response').groupby(data['Year']).size()
firevac_counts = (data['Type'] == 'Fire Response').groupby(data['Year']).size()
evacuation_counts = medevac_counts + firevac_counts
rate_per_evacuation = 100
revenue_per_year = evacuation_counts * rate_per_evacuation
print(revenue_per_year)